### Laboratory of Bioinformatics I (Second Module)

**Main steps for the implementation of the project.**



**Main Aim**: Developemnet of a method for the detection BPTI/Kunitz domain in proteins.

**Methodology**: Hidden Markov Model based on structural alignment.

The workflow of the project is summarized as follows

1.  Selection of a representative set of protein structures from PDB.
2.  Multiple structural alignment with web available tools.
3.  Generation of the Hidden Markow Model for modeling BPTI/Kunitz domain.
4.  Selection of training and testing set from UniProt.
5.  Method optimization and assesment. 

**Selection a representative set of protein structures from PDB.**

For this task we use the advanced search on the PDB web page. It is possible to use different contrains in the search. For the selection of the structures it is important to consider:

*   The resolution
*   The length of the protein
*   External identifiers for BPTI/Kunitz domain (e.g. PFAM, SCOP CATH)

When a list of PDB strutures is returned if it is feasible you should look to the proteins to check if they have a BPTI/Kunitz domain. 
From the PDB web page it is possible to download the using the sequence report it is possible to download and the protein sequence and chec for the possibl redundance in the set of proteins.
To check for the redundancy in the dataset you can use *blastclast* algorithm. Blastclust takes in imput a fasta faile containing a set of sequences adn cluster them according to the level of sequnece identity and coverage between the proteins. The option *-S* and *-L* for the program are used to control the percentage of sequence identity and the coverage respectively.


In [ ]:
#The command sed and awk can be used to clean the csv file returned by the PDB.

# Replace inner comma
sed 's/, /_/g' filename 

# Remove spaces 
sed 's/ //g' filename

# split by comma
awk -F "," '{print $1,$2}'  filename

# After genereting a fasta file run blustclust with sequen identity of 95% and 90% coverage.
blastclust -i seqfile.fasta -o seqfile.clust -S 95 -L 0.9

When the cluster file is returned select a representative structure for each cluster  considering the resolution of the structure and the length of the protein.

**Multiple structural alignment with web available tools.**

After collecting a clean set of protein structures calculate the multiple structure alignment using web available tools such as [PDBeFold](https://www.ebi.ac.uk/msd-srv/ssm/), [PROMALS3D](http://prodata.swmed.edu/promals3d) [mTM-align](https://yanglab.nankai.edu.cn/mTM-align/).
Check the returned alignment and look for the RMSD between pair of structures and the sequence identity. This information can be important to detect possible errors in the initial selection of the BPTI/Kunitz domain proteins.
Finally download the fasta file representing the multiple structure alignment.



**Generation of the Hidden Markow Model for modeling BPTI/Kunitz domain.**

When the alignemnt is returned it is importat to manually check the alignement to look for the conserved residues and the possible errors in the alignement. Ater that you can generate the HMM using *hmmbuild* command from [HMMER](http://hmmer.org/) package. 


In [ ]:
# Generate the HMM
hmmbuild bpti-kunitz.hmm align-bpti-kunitz.fasta

**Selection of training and testing set from UniProt.**

This task is performd using the advanced searc of UniProt database using as a key search the Pfam identifier of the BPTI/Kunitz domain. The search has to be restriictued to the protein in SwissProt the are manually annotated.
With this search you will obtain ~350 protein containing BPTI/Kuntiz domain that will rapresente the positive set. All the remaining proteins in SwissProt can be used as negatives. 

For a fair test of your HMM model you should remove the seqnece in the positive set that share high level of sequence identity with the protein structures collected for generating the model. This task can be performed using *blastpgp* with the option -m 8 checking for sequences with low e-value (column 11) and high sequence identity (column 3).

In [ ]:
# Run trasforming the sequences of the selected structures in database
formatdb -i selected-bpti-kunits.fasta

# Run blastpgp to matche the positive set agains the selected bpti-kunitz.
blastpgp -i positives.fasta -d selected-bpti-kunits.fasta -m 8 -o positives.bl8

Using the output of blastpgp you can select the proteins that should be removed from the set of positives.italicized text

Method optimization and assessment.

When the final benchmark set has been collected a basic testing procedure consist in the implementation of a 2-fold cross-validation test. It consists in splitting positives and negatives in 2 subsets, optimizing the classification threshold on one subset and testing the performance on the other subset.

When the the dataset is splitted in two parts with the same size you can run the hmmsearch on the different subsets. It is better to run the command using the following options:

- --max: turns off all the heuristics for cutting of distantly related proteins
--noali: exclude from the output the alignemnts
--tblout: returns the output in tabular form
- -Z: It is important for normalizing the e-value output

An example of command for macthing a set of sequences (clean_set.fasta) with the HMM model (bpti-kunitz.hmm) is the following:

In [ ]:
# hmmsearch command matching the model (bpti-kunitz.hmm) with sequence in clean_set.fasta

hmmsearch --noali --max --tblout output.txt -Z 1 bpti-kunitz.hmm clean_set.fasta